# Geographische Diagramme

In [1]:
from bokeh.io import output_notebook, show


output_notebook()

Loading BokehJS ...

Es ist oft nützlich, Datensätze mit ihrem realen Kontext in Beziehung setzen zu können. Ihr könnt geografische Daten wie jede andere Art von Daten darstellen, z.B. für [Texas Unemployment example](https://docs.bokeh.org/en/latest/docs/examples/topics/geo/texas_hover_map.html), aber Bokeh bietet auch mehrere spezialisierte Mechanismen zum Zeichnen von Daten in geografischen Koordinaten:

* [TileSource](https://docs.bokeh.org/en/latest/docs/user_guide/topics/geo.html#tile-provider-maps), insbesondere WMTSTileSource: Ermöglicht die Überlagerung von Daten von beliebigen Karten-Tiles-Servern, einschließlich
[Stamen](https://maps.stamen.com), 
[OpenStreetMap](https://www.openstreetmap.org), 
[ESRI](https://www.esri.com/de-de/home), 
und benutzerdefinierte Server.

* [GeoJSONDataSource](https://docs.bokeh.org/en/latest/docs/user_guide/topics/geo.html#geojson-data): Ermöglicht das Lesen von Daten im [GeoJSON](https://geojson.org/)-Format zur Verwendung mit Bokeh-Plots und -Glyphen, ähnlich `ColumnDataSource`.

### WMTS Tile Source

WTMS ist der gebräuchlichste Webstandard für gekachelte Kartendaten, d.H. Karten, die als Bildabschnitte in Standardgröße bereitgestellt werden, aus denen dann die Gesamtkarte mit einer bestimmten Zoomstufe erstellt werden kann. WTMS verwendet das Web Mercator-Format, wobei Entfernungen von Greenwich, England in nördlicher und westlicher Richtung gemessen werden. Dies ist einfach zu berechnen, verzerrt jedoch die globale Form.

Zuerst erstellen wir ein leeres Bokeh-Diagramm, das die USA abdeckt, wobei die Grenzen in Metern angegeben sind:

In [2]:
from bokeh.models import WMTSTileSource
from bokeh.plotting import figure

In [3]:
# web mercator coordinates
USA = x_range, y_range = ((-13884029, -7453304), (2698291, 6455972))

p = figure(
    tools="pan, wheel_zoom",
    x_range=x_range,
    y_range=y_range,
    x_axis_type="mercator",
    y_axis_type="mercator",
)

In [4]:
url = "http://a.basemaps.cartocdn.com/rastertiles/voyager/{Z}/{X}/{Y}.png"
attribution = "Tiles by Carto, under CC BY 3.0. Data by OSM, under ODbL"

p.add_tile(WMTSTileSource(url=url, attribution=attribution))

TileRenderer(id='p1046', ...)

In [5]:
show(p)

Ihr könnt jetzt alles hinzufügen, was ihr normalerweise in einem Bokeh-Diagramm verwenden würdet, sofern ihr im Web Mercator-Format Koordinaten dafür erhaltet, z.B.: 

In [6]:
import numpy as np
import pandas as pd


def wgs84_to_web_mercator(df, lon="lon", lat="lat"):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    df["x"] = df[lon] * (k * np.pi / 180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi / 360.0)) * k
    return df


df = pd.DataFrame(
    dict(
        name=["Austin", "NYC"],
        lon=[-97.7431, -74.0059],
        lat=[30.2672, 40.7128],
    )
)
wgs84_to_web_mercator(df)

,name,lon,lat,x,y
0,Austin,-97.7431,30.2672,-1.088071e+07,3.537942e+06
1,NYC,-74.0059,40.7128,-8.238299e+06,4.970072e+06


In [7]:
p = figure(
    tools="pan, wheel_zoom",
    x_range=x_range,
    y_range=y_range,
    x_axis_type="mercator",
    y_axis_type="mercator",
)

p.add_tile(WMTSTileSource(url=url, attribution=attribution))

p.circle(x=df["x"], y=df["y"], fill_color="orange", size=10)
show(p)